In [1]:
import pandas as pd
import numpy as np
sb_data=pd.read_csv("./data/Shill_Bidding_Dataset.csv",header=0)
sb_data.head()

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


Record ID: Unique identifier of a record in the dataset.

Auction ID: Unique identifier of an auction.

Bidder ID: Unique identifier of a bidder.

Bidder Tendency: A shill bidder participates exclusively in auctions of few sellers rather than a diversified lot. This is a 
collusive act involving the fraudulent seller and an accomplice.
Bidding Ratio: A shill bidder participates more frequently to raise the auction price and attract higher bids from legitimate participants.

Successive Outbidding: A shill bidder successively outbids himself even though he is the current winner to increase the price gradually with small consecutive increments.
Last Bidding: A shill bidder becomes inactive at the last stage of the auction (more than 90\% of the auction duration) to avoid winning the auction.

Auction Bids: Auctions with SB activities tend to have a much higher number of bids than the average of bids in concurrent auctions.

Auction Starting Price: a shill bidder usually offers a small starting price to attract legitimate bidders into the auction.

Early Bidding: A shill bidder tends to bid pretty early in the auction (less than 25\% of the auction duration) to get the attention of auction users.

Winning Ratio: A shill bidder competes in many auctions but hardly wins any auctions.

Auction Duration: How long an auction lasted.

Class: 0 for normal behaviour bidding; 1 for otherwise.

In [2]:
sb_data.describe()

,Record_ID,Auction_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
count,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000
mean,7535.829457,1241.388230,0.142541,0.127670,0.103781,0.463119,0.231606,0.472821,0.430683,0.367731,4.615093,0.106787
std,4364.759137,735.770789,0.197084,0.131530,0.279698,0.380097,0.255252,0.489912,0.380785,0.436573,2.466629,0.308867
min,1.000000,5.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,3778.000000,589.000000,0.027027,0.043478,0.000000,0.047928,0.000000,0.000000,0.026620,0.000000,3.000000,0.000000
50%,7591.000000,1246.000000,0.062500,0.083333,0.000000,0.440937,0.142857,0.000000,0.360104,0.000000,5.000000,0.000000
75%,11277.000000,1867.000000,0.166667,0.166667,0.000000,0.860363,0.454545,0.993593,0.826761,0.851852,7.000000,0.000000
max,15144.000000,2538.000000,1.000000,1.000000,1.000000,0.999900,0.788235,0.999935,0.999900,1.000000,10.000000,1.000000


In [27]:
# data_10day=sb_data[sb_data['Auction_Duration']==10]
data_copy=sb_data.copy()
df=data_copy.to_numpy()
x,y=df[:,3:12],df[:,12]
y=y.astype('int')

print('Original number  of samples: '+str(len(y)))

print(y)


Original number  of samples: 6321
[0 0 0 ... 0 0 0]


In [28]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
print(Counter(y))

Counter({0: 5646, 1: 675})


So, It is clearly visible that the data is unbalanced i.e., number of suspicious activities are limited to 675
Where as number of normal activity is 5646 which is approximately 8-times of suspicious activity.
So, to avoid any kind of biases in the ML model, Oversampling method will be used here.

In [30]:
oversampling = SMOTE()
x, y = oversampling.fit_resample(x, y)
print(Counter(y))

Counter({0: 5646, 1: 5646})


Now, we have oversampled the data and have equal amount of positive and negetive data to work with, to avoid any chances of
bias in learning of the model.

In [32]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import xgboost as xgb


The upcoming block is spling of data into training and testing data and also converting it into Dmatrix which is accepted by 
XGBoost Algorithm

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
train = xgb.DMatrix(X_train, label=y_train)
test = xgb.DMatrix(X_test, label=y_test)
param = {
    'max_depth': 8,
    'eta': 0.03,
    'objective': 'binary:hinge',
    'num_class': 1} 
epochs = 25


In [36]:
model = xgb.train(param, train, epochs)
predictions = model.predict(test)
print('Acuuracy of prediction = %s' % (accuracy_score(y_test, predictions)*100), end="%\n")

Acuuracy of prediction = 99.95573262505533%


In [38]:
dict={1:"Scam/Fraud", 0:"Fair Transaction"}

while True:
    expec=np.random.randint(len(predictions))
    print('\n\nAmount: $%s' % X_test[expec][-1])
    print('Type of transaction: ' + dict[int(y_test[expec])])
    print('Predicted type: '+dict[int(predictions[expec])])
    ans = input("\nContinue? [Y/n]")
    if ans and ans.lower().startswith('n'):
      break



Amount: $1.0
Type of transaction: Fair Transaction
Predicted type: Fair Transaction

Continue? [Y/n]y


Amount: $1.0
Type of transaction: Fair Transaction
Predicted type: Fair Transaction

Continue? [Y/n]y


Amount: $7.0
Type of transaction: Fair Transaction
Predicted type: Fair Transaction

Continue? [Y/n]y


Amount: $1.0
Type of transaction: Scam/Fraud
Predicted type: Scam/Fraud

Continue? [Y/n]n
